In [1]:
import ipl 
import pickle

game = ipl.games.ElMazeGame(3,2)
organism = ipl.Organism()
organism.verbosity = 1

# organism.randomtest = True
organism.configure(game.player_config())

if not organism.randomtest:
  try:
    exprepo = pickle.load(open("organism-exprepo.p", "rb"))
    organism.experience_repo = exprepo
  except FileNotFoundError:
    print('No experience repository file found. Starting from scratch.')

    

In [2]:
game = ipl.games.ElMazeGame(3,2)

organism.verbosity = 1
organism.reset_state()
for iturn, pa in enumerate(action_program):
  print('\nRUNNER: Turn {}'.format(iturn + 1))
  organism.handle_sensor_input(game.sensors())

  oa = organism.choose_action(pa)
  if oa.outcomes and len(oa.outcomes):
    print('Expected outcomes:')
    for outcome in oa.outcomes:
      print('\t{}'.format(outcome))
  else:
    print('(Action has no outcomes)')

  game.act(oa.actuators)
  

print()
print('RUNNER: Predefined action sequence complete.')
organism.handle_sensor_input(game.sensors())

organism.maintenance()

NameError: name 'action_program' is not defined

In [ ]:
game.io_vector_labels()

In [ ]:
print('# learned vectors: {}'.format(len(organism.experience_repo.experiences)))
retval = []

for i in range(0,31):
    a = [int(x) for x in list('{:05b}'.format(i))]
    est = organism.outcome_likelihood_estimator.estimate( ipl.nnplanner.Experience([0, 1, 0, 1, 0], [0, 1, 0, 0],  a))
    retval.append( (a, int(est*100) ) )

retval.sort(key=lambda x: -x[1])
retval

In [ ]:
organism.verbosity = 0
for i in range(0, 20):
    print('Run #{}'.format(i))
    game = ipl.games.ElMazeGame(3,2)    

    organism.reset_state()
    for pa in action_program:
      organism.handle_sensor_input(game.sensors())
      oa = organism.choose_action(pa)
      game.act(oa.actuators)
    

    organism.handle_sensor_input(game.sensors())
    organism.maintenance()
    

In [ ]:
len(organism.experience_repo)

In [32]:
import sklearn.neural_network
neuralnet = sklearn.neural_network.MLPRegressor(
        hidden_layer_sizes=(448)
    )


td = organism.experience_repo.training_data()
neuralnet.fit(td[0], td[1])

retval = []

for i in range(0,31):
    a = [int(x) for x in list('{:05b}'.format(i))]
    est = neuralnet.predict([ [1, 0, 0, 1, 0] + [0, 1, 0, 0] + a ])[0]
    retval.append( (a, int(est*100) ) )

retval.sort(key=lambda x: -x[1])
retval

[([0, 1, 1, 0, 0], 103),
 ([1, 1, 1, 0, 0], 17),
 ([0, 0, 1, 1, 0], 13),
 ([0, 1, 1, 1, 0], 12),
 ([1, 0, 1, 0, 1], 12),
 ([0, 1, 0, 0, 0], 10),
 ([0, 1, 1, 0, 1], 10),
 ([0, 0, 1, 0, 0], 7),
 ([0, 1, 1, 1, 1], 6),
 ([0, 0, 0, 0, 1], 5),
 ([0, 0, 1, 0, 1], 5),
 ([0, 1, 0, 0, 1], 5),
 ([1, 1, 1, 0, 1], 5),
 ([0, 1, 0, 1, 1], 1),
 ([1, 0, 0, 0, 1], 1),
 ([1, 1, 0, 0, 0], 1),
 ([1, 1, 1, 1, 0], 1),
 ([0, 0, 1, 1, 1], 0),
 ([0, 1, 0, 1, 0], 0),
 ([1, 0, 0, 1, 0], 0),
 ([0, 0, 0, 1, 1], -4),
 ([1, 1, 0, 1, 1], -4),
 ([1, 0, 1, 1, 0], -5),
 ([1, 0, 0, 1, 1], -6),
 ([1, 0, 1, 1, 1], -6),
 ([0, 0, 0, 1, 0], -7),
 ([1, 0, 1, 0, 0], -8),
 ([1, 1, 0, 0, 1], -8),
 ([1, 1, 0, 1, 0], -8),
 ([0, 0, 0, 0, 0], -23),
 ([1, 0, 0, 0, 0], -36)]

In [ ]:
len(organism.experience_repo)

In [12]:
organism.outcome_likelihood_estimator.learn(organism.experience_repo)

In [15]:
organism.outcome_likelihood_estimator.estimate(ipl.nnplanner.Experience([1,0,0,0,0], [1,0,0,0], [0,1,1,1,1]))

0.4404706554890489

In [ ]:
begin_exps = [e.training_vector() for e in organism.experience_repo.experiences]
begin_exps[:10]

In [3]:
import sklearn

#(tX, ty) = ([b[0] for b in begin_exps], [b[1] for b in begin_exps])
(tX, ty) = organism.experience_repo.training_data()

nn = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(32, 32, 5), solver='lbfgs')
nn.fit(tX, ty)

nn.predict([[1,0,0,0,0] + [1,0,0,0] + [1,0,0,1,0]])

array([0.8881933])